# Implementation of solution approaches for the robust location-transportation problem
For this project we will consider the following instance of the location-transportation problem with 3 potential warehouse locations and 3 clients:
\begin{align*}
\min \quad &400 x_1+414 x_2+326 x_3+18 z_1+25 z_2+20 z_3\\
&22 y_{11}+33 y_{12}+24 y_{13}+33 y_{21}+ 23 y_{22}+30 y_{23}\\
&20 y_{31}+25 y_{32}+27 y_{33}\\
\text{s.t.}\quad &z_i\leq 800 x_i &\forall i\in[3]\\
&\sum_{j\in[3]} y_{ij}\leq z_i &\forall i\in [3]\\
&\sum_{i\in[3]} y_{ij}\geq \tilde{d}_j &\forall j\in[3]\\
&x\in\{0,1\}^3, z\geq 0, y\geq 0. 
\end{align*}
We will consider that the demand vector $\tilde{d}$ lies in the following polyhedron::
$$D=\left\{d\mid d_1=206+40\delta_1, d_2=274+40\delta_2, d_3=220+40\delta_3, 0\leq \delta_j\leq 1 \text{ for } j\in [3], \sum_{j\in [3]}\delta_j\leq 1.8, \delta_1+\delta_2\leq 1.2 \right\}.$$

The data of the problem is given as: \
$f=[400, 414, 326]$\
$a=[18,25,20]$\
$c=
\begin{bmatrix}
22 &33 &24\\
33 &23 &30\\
20 &25 &27
\end{bmatrix}
$\
$C=[800,800,800]$\
$\bar{d}=[206,274,220]$ \
$\hat{d}=[40,40,40]$ \
$\Gamma_1=1.8$ and $\Gamma_2=1.2$.

The goal of this project is to implement different exact and approximate approaches for the solution of this instance.

We will first start with loading the JuMP and Gurobi packages, and the data of the problem.

In [ ]:
using JuMP
# !!! Change these two lines if you have not installed the Gurobi solver !!!
using Gurobi
solver = Gurobi.Optimizer
# !!! Change these two lines if you have not installed the Gurobi solver !!!
ϵ = 10^-4

In [ ]:
# store problem data in appropriate vectors and matrices

f=[400,414,326] #fixed cost
a=[18,25,20]    #unit capacity cost
c=[22 33 24     #unit transportation cost
    33 23 30
    20 25 27]
C=[800,800,800] #maximum warehouse capacity
d̄=[206,274,220] #nominal demand
d̂=[40,40,40]    #demand deviation
Γ₁=1.8          #uncertainty budget 1
Γ₂=1.2          #uncertainty budget 2
I=1:3           #warehouse index
J=1:3           #customer index

## Question 1: 
Following a similar logic to what was done during the problem session, consider that $y$ are second-stage variables approximated through affine decision rules $y_{ij}(\delta)=\alpha^0_{ij}+\sum_{i,j,k\in[3]}\alpha_{ij}^k\delta_k$
- Plug the definitions of affine decision rules in the model to obtain the affine decision rules approximation.
- Use linear programming duality on the resulting robust problem to obtain a deterministic equivalent formulation with a finite number of variables and constraints.
- Implement this model using the Julia language and the JuMP package to obtain its optimal solution. 
- Interpret the optimal policy obtained, i.e., explain how the recourse variables change their values according to the changes in the demand.

### Decision rule implementation

In [ ]:
# this function constructs and solves the deterministic
# equivalent of the affine decision rule model

# input: none

# output: optimal value and an optimal solution of the affine decision rule model

function apply_affine()
    
    K=1:3           #affine rule parameter index
    
    # declare the model
    ADR = Model(solver)
    
    # silence solver outputs (if you want to see solver logs comment out this line)
    set_optimizer_attribute(ADR, "OutputFlag", 0)
    
    # declare decision variables and add them to the model
    
    # first-stage decision variables
    # x: opening of facility locations
    # z: capacities of facilities
    @variable(ADR, x[i in I],Bin)
    @variable(ADR, z[i in I] >= 0)
    
    # decision-rule variables
    # alpha^0 describes the intersepts
    # alpha describe the slopes
    @variable(ADR, α⁰[i in I,j in J])
    @variable(ADR, α[k in K,i in I,j in J])
    
    # dual variables necessary for reformulating
    # various semi-infinite constraints
    @variable(ADR, u₁ >= 0)
    @variable(ADR, u₂ >= 0)
    @variable(ADR, v[k in K] >= 0)
    @variable(ADR, γ₁[i in I] >= 0)
    @variable(ADR, γ₂[i in I] >= 0)
    @variable(ADR, β[k in K,i in I] >= 0)
    @variable(ADR, λ₁[j in J] <= 0)
    @variable(ADR, λ₂[j in J] <= 0)
    @variable(ADR, μ[k in K,j in J] <= 0)
    @variable(ADR, τ₁[i in I,j in J] <= 0)
    @variable(ADR, τ₂[i in I,j in J] <= 0)
    @variable(ADR, η[k in K,i in I,j in J] <= 0)
    
    
    #### ADD THE CONSTRAINTS ####
    
    #### ADD THE CONSTRAINTS ####
    
    
    #### ADD THE OBJECTIVE FUNCTION ####
    
    #### ADD THE OBJECTIVE FUNCTION ####
    
    
    # solve the stated optimization model
    optimize!(ADR)
    
    # return the objective value and the values of relevant decision variables
    return objective_value(ADR),value.(x),value.(z),value.(α⁰),value.(α)
end

In [ ]:
# call the function that constructs and solves
# the deterministic equivalent of the affine decision rule approximation
(obj,x,z,constant,slope) = apply_affine()

# report the result
println("Objective value is: ", obj)
println("x value: ", x)
println("z value: ", z)

# we report the parameters of each affine function representing y_ij separately
for i in I
    for j in J
        println("y_$(i)_$(j) is expressed as Constant: ", constant[i,j], ", Slope 1: ", slope[1,i,j], ", Slope 2: ", slope[2,i,j], ", Slope 3: ", slope[3,i,j])
    end
end

## Question 2:

Evaluate the quality of the solution obtained using the decision rule approximation. To do so:
- Identify the worst uncertainty realization for each constraint.
- Calculate the dual bound proposed by Hadjiyiannis et al. (2011) by solving a relaxation using the realizations identified previously. 
- According to the dual bound you obtained is the affine decision rule approximation optimal for this instance?

In [ ]:
# this function identifies and returns the worst uncertainty realization for the objective function
# based on the decision rule approximation solution obtained above

# input: none

# output: optimal value and an uncertainty realization 

function separate_objective()
    sep_obj = Model(solver)
    set_optimizer_attribute(sep_obj, "OutputFlag", 0)
    @variable(sep_obj,0<=δ[j in J]<=1)
    @constraint(sep_obj,sum(δ[j] for j in J)<= Γ₁)
    @constraint(sep_obj,δ[1]+δ[2]<= Γ₂)
    @objective(sep_obj, Max,sum(c[i,j]*(constant[i,j]+slope[1,i,j]*δ[1]+slope[2,i,j]*δ[2]+slope[3,i,j]*δ[3]) for i in I,j in J) )
    optimize!(sep_obj)
    return value.(δ)
end

In [ ]:
# this function identifies and returns the worst uncertainty realizations for the warehouse capacity constraints
# based on the decision rule approximation solution obtained above

# input: none

# output: optimal value and identified uncertainty realizations

function separate_capacity()
    scenarios = []
    for i in I
        sep_cap = Model(solver)
        set_optimizer_attribute(sep_cap, "OutputFlag", 0)
        @variable(sep_cap,0<=δ[j in J]<=1)
        @constraint(sep_cap,sum(δ[j] for j in J)<= Γ₁)
        @constraint(sep_cap,δ[1]+δ[2]<= Γ₂)
        #### ADD THE OBJECTIVE FUNCTION ####

        #### ADD THE OBJECTIVE FUNCTION ####
        optimize!(sep_cap)
        println(objective_value(sep_cap))
        println(value.(δ))
        push!(scenarios,value.(δ))
    end
    return scenarios
end

In [ ]:
# this function identifies and returns the worst uncertainty realizations for the demand satisfaction constraints
# based on the decision rule approximation solution obtained above

# input: none

# output: optimal value and identified uncertainty realizations

function separate_demand()
    scenarios = []
    for j in J
        sep_dem = Model(solver)
        set_optimizer_attribute(sep_dem, "OutputFlag", 0)
        @variable(sep_dem,0<=δ[j in J]<=1)
        @constraint(sep_dem,sum(δ[j] for j in J)<= Γ₁)
        @constraint(sep_dem,δ[1]+δ[2]<= Γ₂)
        #### ADD THE OBJECTIVE FUNCTION ####

        #### ADD THE OBJECTIVE FUNCTION ####
        optimize!(sep_dem)
        println(objective_value(sep_dem))
        println(value.(δ))
        push!(scenarios,value.(δ))
    end
    return scenarios
end

In [ ]:
# this function identifies and returns the worst uncertainty realizations for the non-negativity constraints
# based on the decision rule approximation solution obtained above

# input: none

# output: optimal value and identified uncertainty realizations

function separate_nonneg()
    scenarios = []
    for i in I
        for j in J
            sep_nonneg = Model(solver)
            set_optimizer_attribute(sep_nonneg, "OutputFlag", 0)
            @variable(sep_nonneg,0<=δ[j in J]<=1)
            @constraint(sep_nonneg,sum(δ[j] for j in J)<= Γ₁)
            @constraint(sep_nonneg,δ[1]+δ[2]<= Γ₂)
            #### ADD THE OBJECTIVE FUNCTION ####

            #### ADD THE OBJECTIVE FUNCTION ####
            optimize!(sep_nonneg)
            println(objective_value(sep_nonneg))
            println(value.(δ))
            push!(scenarios,value.(δ))
        end
    end
    return scenarios
end

In [ ]:
# this function calculates a dual bound on the optimal value of the adjustable robust optimization problem
# by solving a relaxation based on a discrete set of uncertainty realizations (it can be used with the realizations obtained from the above functions or 
# any other set of discrete realizations)

# input: an array of uncertainty realizations

# output: the calculated dual bound

function solve_dual_bound(dual_scenarios)
    S = length(dual_scenarios)
    println("Number of scenarios: ", S)
    dual = Model(solver)
    set_optimizer_attribute(dual, "OutputFlag", 0)
    @variable(dual, x[i in I],Bin)
    @variable(dual, z[i in I] >= 0)
    @variable(dual, y[1:S,i in I, j in J] >= 0)
    @variable(dual, θ)
    @constraint(dual,[i in I], z[i] <= C[i]x[i])
    #### ADD THE CONSTRAINTS ####

    #### ADD THE CONSTRAINTS ####
    @objective(dual,Min,sum(f[i]*x[i]+a[i]*z[i] for i in I)+θ)
    optimize!(dual)
    return objective_value(dual)
end

In [ ]:
obj_scn = separate_objective()
cap_scn = separate_capacity()
dem_scn = separate_demand()
nonneg_scn = separate_nonneg()
dual_scenarios = unique(vcat([obj_scn], cap_scn, dem_scn, nonneg_scn))
dual_bound = solve_dual_bound(dual_scenarios)

## Question 3:

Implement and compare the constraint and constraint-and-column generation algorithms to solve the problem exactly.
- Compare the performance of both algorithms: number of iterations, solution time, progression of the primal and the dual bound along the iterations.
- Compare the solution obtained from the exact algorithms to that of the affine decision rule approximation. Is the affine decision rule approximation optimal for this instance?

### Functions common to both algorithms

In [ ]:
# this function solves the optimization problem 
# that calculates the maximum deviation over the entire 
# uncertainty set 

# input: none

# output: value of the maximum demand violation

function calculate_max_demand()
    # declare the model
    maxdemand = Model(solver)
    set_optimizer_attribute(maxdemand, "OutputFlag", 0)
    
    # declare the decision variables and add them to the model
    @variable(maxdemand,0<=δ[j in J]<=1)
    
    # add the constraints describing the uncertainty set to the model
    @constraint(maxdemand,sum(δ[j] for j in J)<= Γ₁)
    @constraint(maxdemand,δ[1]+δ[2]<= Γ₂)
    
    # add the objective function
    @objective(maxdemand,Max,sum(d̂[j]*δ[j] for j in J))
    
    # solve model and return its optimal value
    optimize!(maxdemand)
    return objective_value(maxdemand)
end

In [ ]:
# this function solves the separation problem
# needed to calculate the true worst-case value of 
# a given first-stage solution

# since the uncertainty set does not have binary 
# extreme points we use the reformulation based on LP optimality conditions 

# input: (zval,θval): describing the current relaxation solution

# output: dual variables and uncertainty realizations (if any) certifying violation
# of the current solution

function separate(zval,θval)
    
    M = 10000
    
    # declare the model
    SP = Model(solver)
    set_optimizer_attribute(SP, "OutputFlag", 0)
    
    # primal decision variables
    @variable(SP,y[i in I,j in J]>=0)
    
    # dual decision variables
    @variable(SP,λ[j in J]>=0)
    @variable(SP,π[i in I]>=0)
    
    # uncertain parameters
    @variable(SP,0<=δ[j in J]<=1)
    
    # auxiliary variables used to write the big-M constraints
    @variable(SP,u[i in I],Bin)
    @variable(SP,w[j in J],Bin)
    @variable(SP,v[i in I,j in J],Bin)
    
    #### COMPLETE WITH THE CONSTRAINTS DESCRIBING LP OPTIMALITY CONDITIONS ####
    # Primal feasibility

    # Dual feasibility

    # Complementary slackness conditions

    #### COMPLETE WITH THE CONSTRAINTS DESCRIBING LP OPTIMALITY CONDITIONS ####
    
    # add the constraints concerning the uncertain parameters
    @constraint(SP,sum(δ[j] for j in J)<=Γ₁)
    @constraint(SP,δ[1]+δ[2]<=Γ₂)
    
    # add the objective function
    @objective(SP,Max,sum(c[i,j]*y[i,j] for i in I, j in J))
    
    # solve model
    optimize!(SP)
    
    # if there is a violation obtained then return optimal dual variables
    # and uncertain parameters
    # ohterwise return empty
    if objective_value(SP) >= θval+ϵ
        return (value.(δ),value.(λ),value.(π),objective_value(SP))
    else
        return ([],[],[],objective_value(SP))
    end
end

### Constraint-and-Column Generation (CCG) implementation

In [ ]:
# this function constructs the initial relaxation
# of the constraint-and-column generation algorithm

# input: dmax: the value of maximum demand violation

# output: a relaxation model, and the reference to variables to be added later

function construct_ccg_master(dmax)
    # declare the model
    master = Model(solver)
    set_optimizer_attribute(master, "OutputFlag", 0)
    
    # declare and add first-stage decision variables
    # an initial bound of 0 on θ is easily established by  
    # non-negativity of costs and variables
    @variable(master,θ>=0) 
    @variable(master,x[i in I],Bin)
    @variable(master,z[i in I]>=0)
    
    # add first-stage constraints: the second constraint
    # ensures that we have relatively complete recourse
    @constraint(master,[i in I],z[i]<= C[i]*x[i])
    @constraint(master,sum(z[i] for i in I)>=sum(d̄)+dmax)   
    
    # add the objective function
    @objective(master,Min,sum(f[i]*x[i]+a[i]*z[i] for i in I)+θ)
    
    # create a reference to the second-stage variables
    # that will be instantiated later
    # y is an array of variable references with each variable 
    # reference having two dimensions
    y = Array{VariableRef,2}[]
    
    # return the model and the reference to the y variables
    return master,y
end

In [ ]:
# this function is meant to execute the necessary
# changes to the constraint-and-column generation 
# relaxation each time a violated realization is found

# input: ξ is an uncertainty realization, rmaster is the current CCG relaxation
#        yref is the reference of y variables array and numiter is the current iteration count

# output: none (changes should be made directly to the model rmaster)

function add_ccg_scenario(ξ,rmaster,yref,numiter)
    # create and add a new set of variables indexed by i,j to rmaster
    # these variables are pushed to the yref array at the same time
    push!(yref,@variable(rmaster,[i in I,j in J], lower_bound = 0))
    
    # name the newly created variables 
    for i in I
        for j in J
            # end points to the last element of an array
            set_name(yref[end][i,j], "y_$numiter,$i,$j")
        end
    end
    
    #### COMPLETE WITH THE CONSTRAINTS TO BE ADDED TO THE CURRENT RELAXATION AT EACH ITERATION ####
    
    #### COMPLETE WITH THE CONSTRAINTS TO BE ADDED TO THE CURRENT RELAXATION AT EACH ITERATION ####
end

In [ ]:
# this function executes the constraint-and-column generation algorithm
# with the help of other functions that it calls in the process

# input: none

# output: none (simply prints the optimal value and an optimal solution)

function apply_ccg()
    
    # calculate the value dmax
    dmax = calculate_max_demand()
    
    # construct the initial CCG relaxation
    rmaster,yref = construct_ccg_master(dmax)
    
    # initialize iteration count and bounds
    numiter = 1
    UB = +Inf
    LB = -Inf
    
    while true
        # solve the current CCG relaxation
        optimize!(rmaster)
        
        # recover the current optimal z and θ values
        # note the use of rmaster[:name_of_variable]
        # this is used to acces the variables of a JuMP 
        # model 
        # the function value is broadcast to all 
        # variables z[i] for i in I with the dot 
        (z,θ) = (value.(rmaster[:z]),value(rmaster[:θ]))
        
        # current master value can be recovered as a lower bound
        LB = objective_value(rmaster)
        
        println("master obj: $LB")
        println("estimated value function: $θ")
        
        # separate the current master solution 
        (ξ,λ,π,θ̄) = separate(z,θ)
        
        println("true value function: $θ̄")
        println("violated scenario: $ξ")
        
        # if a violated uncertainty realization is found 
        # then add the necessary variables and constraints 
        # to the master by calling the function add_
        # ccg_scenario() update the upper bound if necessary
        # otherwise conclude convergence (set UB=LB)
        if ξ!=[]
            add_ccg_scenario(ξ,rmaster,yref,numiter)
            UB = min(UB,LB-θ+θ̄)
        else 
            UB = LB
        end
        
        println("Upper bound is: $UB")
        println("Lower bounds is: $LB")
        
        # check the convergence of bounds as an early stopping criterion
        if 100*(UB-LB)/LB <= ϵ  break end
        print(rmaster)
        numiter += 1
    end
    
    println("Optimality achieved")
    
    # solve master one last time to recover an optimal solution
    # report this solution
    optimize!(rmaster)
    println(objective_value(rmaster))
    println(value.(rmaster[:x]))
    println(value.(rmaster[:z]))
    println(value(rmaster[:θ]))
end

In [ ]:
apply_ccg()

### Constraint generation algorithm implementation

In [ ]:
# this function constructs the initial relaxation
# of the constraint generation algorithm

# input: dmax: the value of maximum demand violation

# output: a master model

function construct_cg_master(dmax)
    
    # declare the model
    master = Model(solver)
    set_optimizer_attribute(master, "OutputFlag", 0)
    
    # declare and add first-stage decision variables
    # an initial bound of 0 on θ is easily established by  
    # non-negativity of costs and variables
    @variable(master, θ >= 0)
    @variable(master, x[i in I],Bin)
    @variable(master, z[i in I] >= 0)
    
    # add first-stage constraints: the second constraint
    # ensures that we have relatively complete recourse
    @constraint(master,[i in I], z[i] <= C[i]*x[i])
    @constraint(master, sum(z[i] for i in I) >= sum(d̄)+dmax)  
    
    # add the objective function
    @objective(master,Min, sum(f[i]*x[i]+a[i]*z[i] for i in I)+θ)
    
    # return the created model
    return master
end

In [ ]:
# this function is meant to execute the necessary
# changes to the constraint generation 
# master model each time a violated constraint is found

# input: δ is an uncertainty realization, (λ, π) are dual variables
#        rmaster is the current constraint generation relaxation

# output: none (changes should be made directly to the model rmaster)


function add_cg_scenario(δ,λ,π,rmaster)
    ####  COMPLETE WITH THE CONSTRAINTS TO BE ADDED TO THE CURRENT RELAXATION AT EACH ITERATION ####
    
    ####  COMPLETE WITH THE CONSTRAINTS TO BE ADDED TO THE CURRENT RELAXATION AT EACH ITERATION ####
end

In [ ]:
# this function executes the constraint generation algorithm
# with the help of other functions that it calls in the process

# input: none

# output: none (simply prints the optimal value and an optimal solution)

function apply_cg()
    
    # calculate the value dmax
    dmax = calculate_max_demand()
    
    # construct the initial constraint generation master model
    rmaster = construct_cg_master(dmax)
    
    # initialize iteration count and bounds
    numiter = 1
    UB = +Inf
    LB = -Inf
    
    while true
        
        # solve the current constraint generation master model
        optimize!(rmaster)
        
        # recover the current optimal z and θ values
        (z,θ) = (value.(rmaster[:z]),value(rmaster[:θ]))
        LB = objective_value(rmaster)
        
        println("master obj: $LB")
        println("estimated value function: $θ")
        
        # separate the current master solution 
        (ξ,λ,π,θ̄) = separate(z,θ)
        
        println("true value function: $θ̄")
                     
        # if a violated cut is found 
        # then add the necessary constraints 
        # to the master by calling the function add_cg_scenario() 
        # update the upper bound if necessary
        # otherwise conclude convergence (set UB=LB)
        if ξ!=[]
            println("model modified")
            add_cg_scenario(ξ,λ,π,rmaster)
            UB = min(UB,LB-θ+θ̄)
        else 
            UB = LB
        end
        
        println("Upper bound is: $UB")
        println("Lower bounds is: $LB")
        
        # check the convergence of bounds as an early stopping criterion
        if 100*(UB-LB)/LB <= ϵ 
            break 
        end
        print(rmaster)
        numiter += 1
    end
    
    println("Optimality achieved")
    
    # solve master one last time to recover an optimal solution
    # report this solution
    optimize!(rmaster)
    println(objective_value(rmaster))
    println(value.(rmaster[:x]))
    println(value.(rmaster[:z]))
    println(value.(rmaster[:θ]))
end

In [ ]:
apply_cg()